# utils

> Reads the data and formats it.

In [ ]:
#| default_exp utils
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from sys import platform
from typing import Tuple
from pathlib import Path
from importlib.resources import files
import pandas as pd

In [ ]:
#| export
DATA_PATH = Path("..") / Path("Datasets")
if platform in ["linux", "linux2"]:
    DATA_PATH = Path("../../Datasets")

GENERAL_RESULTS_FOLDER = Path("Results")
PDF_FOLDER = Path("pdfs")
DASH = "_"

PARAMETERS = "parameters"
VARIABLE_RHO_NOTEBOOK_PATH_NO_EXTENSION = "variable_rho"
CONSTANT_RHO_NOTEBOOK_PATH_NO_EXTENSION = "constant_rho"
ACCUMULATED_CONSTANT_RO_DATA_PATH = "accumulated_constant_ro_data.npy"
PREDICTING_FHAT_ACCURACIES_DATA_PATH = "predicting_fhat_accuracies_data.npy"
PREDICTING_FHAT_NOTEBOOK_HTML_PATH = "predicting_fhat_notebook.html"
PREDICTING_FHAT_SEE_RESULTS_NO_EXTENSION = "predicting_fhat_see_results"
ACCUMULATED_DATA_SEE_RESULTS_NO_EXTENSION = "accumulated_data_see_results"

We start by loading the dataset from the FFSA 1989 survey --- only beginning drivers. The data has 6,333 insurees. Each observation has 58 variables:

0. (AG25) whether the insuree is below 25 (1) or not (2)
1. (AGE) the age of the insuree in years
2. (AGEC) the age of the driver in years (9 if the driver was the insuree)
3. (ALLE) German car: 1 yes, 0 no
4. (AMER) American car: 1 yes, 0 no
5. (ANCC) how old the driver license is (1 for less than 3, 2 for more, 9 incomplete, 0 missing)
6. (CFDO) the nature of the deductible (1 fixed, 2 proportional, 0 no deductible)
7. (CFVO) whether the theft guarantee comes with a deductible (1 yes, 2 no, 9 not insured against theft)
8. (CGAR) compulsory insurance (0 is basic, 1 with theft, 2 with fire, 3 =1+2, 4 with windows, 5=1+4, 6=2+4, 7=1+2+4)
9. (CGDO) comprehensive insurance (0 none, 1 no deductible, 2 with deductible except if collision, 3 with deductible always, 4 others)
10. (CIDA) percentage at fault for claims smaller than 25,000FF: 0 = 0%, 1=25%, 2=50%, 3=75%, 4=100%, 9=incomplete, 99=missing.
11. (CITR) Citroën car: 1 yes, 0 no
12. (CODC) driver is the insured (1 yes, 2 no, 9 ncomplete, 0 missing)
13. (CODE) accident (0 if none, 30+ if an accident)
14. (CRES) responsibility (1 total, 2 none, 3 partial, 0 missing, 9 incomplete)
15. (CSOC) insurance company code
16. (DCIR) year the car started circulating
17. (DEPT) département where the car is registered
18. (DNAI) year of birth of insuree
19. (FDOS) claim processing code (1 done, 2 in process, 0 missing)
20. (FRAN) 1 if French car but not Citroën-Peugeot-Renault, 0 otherwise
21. (GROUP) group: from 1 to 6
22. (GSIN) guarantee activated by claim (1 compulsory or comprehensive, 2 theft, 3 fire, 4 windows, 9 missing, 0 ???)
23. (HOM) 1 if insuree is male, otherwise 0
24. (INDACCI) 1 if no accident at fault, 0 otherwise
25. (ITAL) Italian car: 1 yes, 0 no
26. (KVEH) number of cars in claim, including the insuree's (0 missing, 9 incomplete)
27. (LIEU) département of claim (99 incomplete, 0 missing)
28. (MFDO) deductible as percentage of claim value, or as amount if fixed (0 missing)
29. (MFVO) deductible for theft (0 missing)
30. (MIDA) reimbursement for claim (0 missing)
31. (MODR) how the claim was paid (1 interinsurers, 2 no, 0 missing)
32. (MSIN) the total amount of the claim (0 missing)
33. (MTRC) the amount covered by the compulsory guarantee (0 missing)
34. (NATS) the nature of the claim (1 material, 2 personal or both, 0 missing)
35. (NBACCI) the number of claims at fault
36. (NORD) an identifier
37. (NSEQ) another one
38. (PBAS) the basic premium (= [45]/[51]*100)
39. (PEUG) Peugeot car: 1 yes, 0 no
40. (PFIS) car fiscal horsepowers (3 a 8)
41. (PNET) total premium, after bonus/malus, net of taxes
42. (POID) percentage of the year covered by insurance
43. (PROF) occupation (1 shopowner or crafts, 2 salaried, 3 civil servant, 4 farmer, 5 others, 6 retired, 7 other w/o a job, 8 unknown)
44. (QSIN) number of claims in the year (9 missing)
45. (RCDR) premium for compulsory coverage after bonus, net of taxes
46. (REG) region (1 is Bourgogne-Franche Comté, 2 is Bassin Parisien W, 3 is PACA, 4 is Centre W, 5 is Grand W, 6 is SW, 7 is Ile de France, 8 is Rhône-Alpes, 9 is Est, 10 is Nord)
47. (RENA) Renault car: 1 yes, 0 no
48. (SCLA) class of car (how expensive, 0 to 4)
49. (SEXE) gender (1 male, 2 female)
50. (SGRP) real group of the car (4 to 13)
51. (TMAJ) bonus/malus (basis 100)
52. (TRACIR) based on [16]; is 11 if car started circulating before 1978, 10 if in 1978 or 1979, else 1989-[16]
53. (TRAGE) age category, based on [2]. Is 0 if [2] is 18 or 19, 1 to 4 if 20 to 23, 5 if 24 or 25, 6 if 26 to 30, 7 if 31 to 45, then 8.
54. (TRAPBAS) category of basic premium, based on [38]. 1 to 6 with breaks at 1,100; 1,400; 1,800; 2,300; and 2,900
55. (TRES) percentage at fault if shared responsibility (0 missing)
56. (USAG) main usage of car; 1: work, 2: home-to-work, 3: private use, 4: unknown


In [ ]:
#| export

def read_data(dataset: str # "j88" or "jpropre"
             ) -> pd.DataFrame: # a clean data frame
    """
    Reads the dataset selected and returns a clean Pandas dataframe
    """
    data_file = f"{dataset}.txt"
    names_vars = [
        'Age below 25', 'Age', 'Age of driver', 'German car', 'American car', 'Age of License',
                  'Type of deductible', 'Theft deductible', 'Compulsory coverage', 'Comprehensive coverage',
                  'Responsibility', 'Citroën', 'Driver is insuree', 'No accident', 
                    'Responsibility code', 'Company',
                  'Date car', 'Department registered', 'Year birth insuree', 'Claim processing',
                  'Other french car', 'Group', 'Guarantee in claim', 'Male', 'Not at fault', 
                  'Italian car', 'Number cars in claim', 'Department claim', 'Deductible damages',
                  'Deductible theft', 'Reimbursement', 'Settlement', 'Total cost', 'Compulsory cost', 'Nature claim', 
                  'Number claims at fault', 'Identifier 1', 'Identifier 2', 'Basic premium',
                  'Peugeot', 'Fiscal HP', 'Total premium', 'Duration', 'Occupation', 'Number claims',
                  'Compulsory premium', 'Region', 'Renault', 'Class car', 'Gender', 'Real group',
                  'Bonus Malus', 'Age category car', 'Age category insuree', 'Basic premium category', 
                  'Shared responsibility', 'Car use', 'Zone'
                 ]
    data_dir = Path(__file__).parent / 'Datasets'
    data = pd.read_csv(data_dir / data_file, delimiter = ' ', header=None)
    data.columns = names_vars
    # we change the types of the non-categorical variables
    for float_col in ['Deductible damages', 'Deductible theft', 'Reimbursement', 'Total cost', 'Compulsory cost',
                     'Total premium', 'Bonus Malus']:
        data[float_col]=data[float_col].astype(float)
    return data

In [ ]:
#| export

def select_variables(data: pd.DataFrame # the Pandas data frame
                    ) -> Tuple[pd.Series, pd.Series, pd.Series, pd.DataFrame]: # `y1, y2, w, X`
    """ returns the `y` variables, the weights, and the `X` covariates"""
    y1 = 1*(data['Comprehensive coverage'] > 0)
    y2 = 1*(data['Not at fault'] == 0)
    w = data['Duration']

    X = data[['Group', 'Male', 'Occupation', 'Region', 'Renault', 'Age category car',
           'Age category insuree', 'Car use', 'Zone']].astype('category')

    return y1, y2, w, X

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()